# 1. Импорт необходимых библиотек

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import numpy as np
import random
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

# 2. Фиксация random_state для воспроизводимости

In [2]:
# Фиксируем все основные "сиды": Python, NumPy и PyTorch (CPU/GPU).
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(42)

#  3. Загрузка и подготовка данных (CIFAR100)

### **Шаги**:
### 1. Определим `transform` (преобразования данных).
### 2. Загрузим train и test части датасета.
### 3. Отделим валидационную часть от обучающей.

In [3]:
# Трансформации (аугментации) для обучающей выборки
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

# Трансформации для валидации/теста (без аугментаций, только нормализация)
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

# Загрузка train/test датасетов
train_dataset_full = torchvision.datasets.CIFAR100(
    root='./data',
    train=True,
    download=True,
    transform=train_transform
)

test_dataset = torchvision.datasets.CIFAR100(
    root='./data',
    train=False,
    download=True,
    transform=test_transform
)

# Разделение на train и validation
train_size = int(0.9 * len(train_dataset_full))  # 90% под train
val_size = len(train_dataset_full) - train_size   # 10% под validation

train_dataset, val_dataset = random_split(train_dataset_full, [train_size, val_size],
                                          generator=torch.Generator().manual_seed(42))

# При разделении на валидационную часть используем другие трансформации (без аугментаций)
val_dataset.dataset.transform = test_transform

# Создание DataLoader'ов
BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Посмотрим на размеры полученных выборок
print(f'Размер обучающей выборки: {len(train_dataset)}')
print(f'Размер валидационной выборки: {len(val_dataset)}')
print(f'Размер тестовой выборки: {len(test_dataset)}')

Files already downloaded and verified
Files already downloaded and verified
Размер обучающей выборки: 45000
Размер валидационной выборки: 5000
Размер тестовой выборки: 10000


# 4. Определение архитектуры AlexNet

### Для задачи CIFAR100 (размер входного изображения 32x32) мы адаптируем свёрточные слои и пулы  под меньшее разрешение, сохраняя общую логику слоёв.

In [4]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=100):
        super(AlexNet, self).__init__()

        # Архитектура свёрточных слоёв (адаптированная под 32x32)
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),   # выход 16x16

            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),   # выход 8x8

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)    # выход 4x4
        )

        # Полносвязные слои
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 4 * 4, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.classifier(x)
        return x

# Проверим, что модель создаётся и вычисляет выход нужной размерности
model_test = AlexNet(num_classes=100)
x_test = torch.randn(2, 3, 32, 32)
out_test = model_test(x_test)
print("Выход тестового батча:", out_test.shape)

Выход тестового батча: torch.Size([2, 100])



# 5. Обучение модели

### Определим функцию тренировки на одну эпоху и функцию валидации. Также инициализируем оптимизатор, зададим количество эпох и будем отслеживать динамику loss на train/val.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Используется устройство: {device}')

model = AlexNet(num_classes=100).to(device)

# Параметры обучения
lr = 0.001
EPOCHS = 10

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

def train_one_epoch(model, optimizer, criterion, dataloader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

def validate(model, criterion, dataloader, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)

            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

train_losses = []
val_losses = []
train_accs = []
val_accs = []

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_one_epoch(model, optimizer, criterion, train_loader, device)
    val_loss, val_acc = validate(model, criterion, val_loader, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    print(f"Эпоха [{epoch}/{EPOCHS}]: "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

# Визуализируем кривые обучения: loss и accuracy

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title('Loss во время обучения')
plt.xlabel('Эпоха')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Acc')
plt.plot(val_accs, label='Val Acc')
plt.title('Accuracy во время обучения')
plt.xlabel('Эпоха')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()

Используется устройство: cuda
Эпоха [1/10]: Train Loss: 4.2167, Train Acc: 4.66% | Val Loss: 3.7874, Val Acc: 9.18%
Эпоха [2/10]: Train Loss: 3.5955, Train Acc: 13.64% | Val Loss: 3.3516, Val Acc: 18.92%
Эпоха [3/10]: Train Loss: 3.2528, Train Acc: 19.52% | Val Loss: 3.0648, Val Acc: 24.50%
Эпоха [4/10]: Train Loss: 2.9905, Train Acc: 24.94% | Val Loss: 2.9015, Val Acc: 27.24%
Эпоха [5/10]: Train Loss: 2.7619, Train Acc: 29.11% | Val Loss: 2.7203, Val Acc: 30.26%
Эпоха [6/10]: Train Loss: 2.5924, Train Acc: 32.61% | Val Loss: 2.5490, Val Acc: 33.64%
Эпоха [7/10]: Train Loss: 2.4386, Train Acc: 35.81% | Val Loss: 2.4422, Val Acc: 37.38%
Эпоха [8/10]: Train Loss: 2.3020, Train Acc: 38.36% | Val Loss: 2.4183, Val Acc: 36.76%


In [ ]:
# Проверим итоговое качество на тестовой выборке (accuracy).
test_loss, test_acc = validate(model, criterion, test_loader, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")





### Данную модель можно улучшать, меняя гиперпараметры (оптимизатор, lr, регуляризацию, аугментации и т.д.). Но основная цель — воспроизвести базовую архитектуру и продемонстрировать корректную работу — выполнена.